In [1]:
from fastai.text import *
import pandas as pd
from path import Path as path
import re

In [2]:
reg = "(intp|intj|infp|infj|istj|istp|isfj|isfp|entj|entp|enfj|enfp|estp|estj|esfp|esfj)"

In [5]:
def cleanup(original, n = 1200):
    texts = original['text']
    texts2 = [re.sub(reg,"",t, flags=re.I) for t in texts]
    texts3 = [t for t in texts2 if len(str(t).split(" ")) < 1000]
    newdf = pd.DataFrame()
    newdf['subreddit'] = [original['subreddit'][0]]*len(texts3)
    newdf['text'] = texts3
    return newdf.sample(n)

In [6]:
path2 = path("model/").mkdir_p()
df = pd.DataFrame()
for f in path("./").files():
    if f.ext == ".csv":
        tmp = pd.read_csv(f,index_col = 0).dropna()
        df = df.append(cleanup(tmp))

In [7]:
print(len(df))

19200


In [8]:
df.to_csv("model/mbti.csv", index=None)

In [9]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path2, 'mbti.csv', num_workers = 0)
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path2, 'mbti.csv', vocab=data_lm.train_ds.vocab, bs=32, num_workers = 0)

In [10]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [11]:
data_lm = load_data(path2, 'data_lm_export.pkl')
data_clas = load_data(path2, 'data_clas_export.pkl', bs=16)

In [12]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.335892,4.062385,0.254092,05:53


In [10]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.040785,3.958968,0.268093,09:00


In [11]:
learn.predict("This is a review about", n_words=10)

'This is a review about you ending up being a 27 years old ❤ ️'

In [12]:
learn.save_encoder('ft_enc')

In [13]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [14]:
data_clas.show_batch()

text,target
xxbos xxmaj by xxmaj leona xxmaj xxunk & xxmaj mark xxmaj hunziker \r\r \n xxmaj dominant for xxup istj and xxup isfj \r\r \n xxmaj auxiliary for xxup estj and xxup esfj \r\r \n xxmaj we seek to present a picture of the “ pure ” xxmaj introverted xxmaj sensing that we would see if we could carefully remove it from its natural state where it is,istj
"xxbos xxmaj so unlike you , i 'm a respond to each point mentioned rather than skim over and insist that her xxup entp boyfriend is just misunderstood and maybe she 's just too controlling because "" she 's an xxup estj "" . xxmaj it 's going to be long . \r\r \n ​ \r\r \n xxmaj yes , i 'm open to it . xxmaj so",estj
"xxbos a couple thoughts : xxmaj when it comes to backing off and “ testing ” someone in your life , this obviously depends on the situation . xxmaj if you ’re simply missing the things that person does to show they care because they ’re different than the things you do to show you care , then maybe testing them is n’t going to work . xxmaj read about",infj
"xxbos xxmaj hi all , xxup intj male in peace here ! i married my girlfriend of just under 5 years last month , and out of curiosity persuaded her to do an xxup mbti test , having found i learnt a lot more about myself and how my personality comes across to everyone else by doing so . xxmaj having found out how diametrically opposed our respective personality types",esfj
"xxbos xxup enfj here : \r\r \n xxmaj yeah , so i ’m probably not going to say anything you have n’t already heard or read . i ’ve had an xxunk in my life for a while now , and i can easily say that few things have ever made me so happy :) i do n’t really know your xxunk situation , and i can only offer",enfj


In [15]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.652555,2.621198,0.130695,10:12


In [16]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.631979,2.589590,0.147878,12:26


In [17]:
learn.export("model")

In [18]:
learn.save("model1")

In [19]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: CUDA out of memory. Tried to allocate 40.50 MiB (GPU 0; 4.00 GiB total capacity; 2.98 GiB already allocated; 11.01 MiB free; 40.84 MiB cached)